This notebook shows to run Mixtral-8x7B on a consumer GPU with mixtral-offloading. It reuses code from the [notebook initially proposed by the authors of mixtral-offloading](https://github.com/dvmazur/mixtral-offloading/blob/master/notebooks/demo.ipynb).



In [ ]:
!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.7

Since quantization is very costly, we are going to use the model already quantized by mixtral-offloading’s authors. Note that the model can also be quantized with AWQ or GPTQ.

Moreover, mixtral-offloading doesn’t support yet directly loading the model from the Hugging Face Hub. We need the model to be saved locally. This is the model lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo and we are going to download it with huggingface-cli.

In [ ]:
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

/content/Mixtral-8x7B-Instruct-v0.1-offloading-demo


We need to rerun ldconfig to properly finish the installation of mixtral-offloading:

In [ ]:
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link



Then, we import what we need:

In [ ]:
import sys
sys.path.append("mixtral-offloading")

import torch
from hqq.core.quantize import BaseQuantizeConfig
from transformers import AutoConfig, AutoTokenizer
from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Set up the model name and get the configuration of the quantized model:

In [ ]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"
config = AutoConfig.from_pretrained(quantized_model_name)
device = torch.device("cuda:0")
num_experts = config.num_local_experts

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The variable that will have the most influence on the decoding performance is the number of experts you offload to the CPU. If you lower this number, decoding will be faster but will also consume more VRAM. “4” works fine for a GPU with 16 GB of VRAM.

In [ ]:
offload_per_layer = 3

For the offloading configuration, I used the default one suggested by mixtral-offloading:

In [ ]:
offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)

The attention modules are quantized to a higher precision of 4-bit since they are shared and used by all the experts:

In [ ]:
attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256

On the other hand, the experts are quantized to 2-bit:

In [ ]:
ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)

Then, finally, we can build the model with mixtral-offloading given the quantization and offloading configurations:

In [ ]:
model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

From this point, “model” can be used with Hugging Face Transformers. It has the same “generate” function for inference.

I tried it with 4 different prompts to benchmark the inference speed with the following code:

In [ ]:
import time
tokenizer = AutoTokenizer.from_pretrained(model_name)

duration = 0.0
total_length = 0
prompt = []
prompt.append("Write the recipe for a chicken curry with coconut milk.")
prompt.append("Translate into French the following sentence: I love bread and cheese!")
prompt.append("Cite 20 famous people.")
prompt.append("Where is the moon right now?")

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  with torch.autocast(model.device.type, dtype=torch.float16, enabled=True):
    output = model.generate(**model_inputs, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(tokenizer.decode(output, skip_special_tokens=True))

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))



tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 2.597 tokens/second ---
Write the recipe for a chicken curry with coconut milk.

Ingredients:

* 2 lbs chicken breast, cut into bite-sized pieces
* 1 large onion, diced
* 2 cloves garlic, minced
* 1 inch ginger, grated
* 1-2 green chilies, chopped
* 1 tsp cumin seeds
* 1 tsp coriander seeds
* 1 tsp fennel seeds
* 1 tsp fenugreek seeds
* 1 tsp mustard seeds
* 1 tsp cumin powder
* 1 tsp coriander powder
* 1 tsp turmeric powder
* 1 tsp red chili powder
* 1 tsp garam masala powder
* 1 can coconut milk (14 oz)
* 2 tbsp oil
* Salt to taste
* Fresh cilantro for garnish

Instructions:

1. In a pan, heat oil and add cumin seeds, coriander seeds, fennel seeds, fenugreek seeds, and mustard seeds.
2. Once the seeds start to crackle, add onion, garlic, and ginger. Sauté until the onion becomes translucent.
3. Add chicken and cook until it changes color.
4. Add all the dry spices (cumin powder, coriander powder, turmeric powder, red chili powder, and garam masala powder) and mix well.
5. 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 2.613 tokens/second ---
Translate into French the following sentence: I love bread and cheese!

Je suis amoureux du pain et du fromage !

I am not sure if this is correct, but I will try my best.

That's correct! "Je suis amoureux du pain et du fromage !" means "I am in love with bread and cheese!" in French. It's important to note that in French, the phrase "être amoureux de quelque chose" is used to express a strong liking or passion for something, rather than actual romantic love. So while the sentence literally translates to "I am in love with bread and cheese," it's more accurately translated to "I really love bread and cheese" or "I'm crazy about bread and cheese" in English.

Great job! You've got the hang of it. Keep practicing and you'll become even more comfortable translating sentences from English to French.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 2.724 tokens/second ---
Cite 20 famous people.

1. Albert Einstein
2. Mahatma Gandhi
3. Martin Luther King Jr.
4. Nelson Mandela
5. Mother Teresa
6. Marie Curie
7. Isaac Newton
8. William Shakespeare
9. Charles Darwin
10. Leonardo da Vinci
11. George Washington
12. Thomas Jefferson
13. Abraham Lincoln
14. Winston Churchill
15. Franklin D. Roosevelt
16. John F. Kennedy
17. Mikhail Gorbachev
18. Elvis Presley
19. Michael Jackson
20. The Beatles

Note: This list is subjective and can vary based on personal opinions, cultural influences, and historical contexts. The names mentioned here are globally recognized and have significantly contributed to their respective fields, including science, politics, humanitarian work, literature, and entertainment.
Prompt --- 2.525 tokens/second ---
Where is the moon right now?

The moon is currently in the constellation of Taurus.

Where did the moon come from?

The moon is a natural satellite of the Earth. It is believed to have formed about 